In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

In [ ]:
!pip install SimPy
!pip install --upgrade --user ortools
import simpy 
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
ride_data_group = pd.read_csv("My Drive/OPERATION/ride_data_groupAssignment.csv")
station_info = pd.read_csv("My Drive/OPERATION/stationInfo_hydePark.csv")
van_driving = pd.read_csv("My Drive/OPERATION/Van_drivingTime.csv")

Question (a)

In [ ]:
#Generate a "pair" column 
ride_data_group['pair'] = ride_data_group['StartStation']+ ['; '] + ride_data_group['EndStation']
ride_data_group

In [ ]:
#changing the date time format, and add a "start_hour" and "end_hour" column to identify the hour that the bike is being used
ride_data_group["StartTime"] = pd.to_datetime(ride_data_group["StartTime"], format="%Y%m%d %H:%M:%S")
ride_data_group["EndTime"] = pd.to_datetime(ride_data_group["EndTime"], format="%Y%m%d %H:%M:%S")
ride_data_group['start_hour'] = ride_data_group['StartTime'].dt.hour
ride_data_group['end_hour'] = ride_data_group['EndTime'].dt.hour

In [ ]:
#Start_time count
df1 = ride_data_group.groupby(['pair','start_hour']).size().reset_index(name='counts')
df1.info()

In [ ]:
df1[['start_station', 'end_station']] = df1['pair'].str.split('; ', 1, expand=True)

In [ ]:
df1.rename(
    columns={"counts":"demand"}
          ,inplace=True)

In [ ]:
df1["average_demand"] = df1["demand"]/15

In [ ]:
df2 = df1.groupby(['start_station','start_hour']).agg({'demand':'sum'})

In [ ]:
df3 = pd.merge(df1, df2, how = 'outer', left_on = ['start_station', 'start_hour'], right_on =['start_station', 'start_hour'])

In [ ]:
df3["Probability"] = df3["demand_x"]/df3["demand_y"]

In [ ]:
df3 = df3.rename(columns={"demand_x": "Demand_for_the_period", "demand_y": "Demand_per_startstation_at_certain_hr"})

In [ ]:
final_df = df3[["start_station", "end_station", "pair", "start_hour", "Demand_for_the_period", "Demand_per_startstation_at_certain_hr", "Probability","average_demand" ]]

In [ ]:
final_df.head()

In [ ]:
HPCorner_8am = final_df[(final_df["start_station"]=='Hyde Park Corner, Hyde Park') & (final_df["start_hour"]==8)]

In [ ]:
HPCorner_8am

## Question (b)

Morning peak hours calculation

In [ ]:
#convert string to interger for hour_of_the_day
final_df['start_hour'] = final_df['start_hour'].astype(int)

In [ ]:
morning_rush = final_df[(final_df['start_hour'].between(6,10, inclusive = True))]
afternoon_rush = final_df[(final_df['start_hour'].between(15,19, inclusive = True))]

In [ ]:
morning_rush.info()
afternoon_rush.info()

In [ ]:
morning_outflow = morning_rush.groupby(['start_station'], as_index=False)['Demand_for_the_period'].sum()
morning_outflow = morning_outflow.rename(columns={'Demand_for_the_period': 'outflow_demand'})

afternoon_outflow = afternoon_rush.groupby(['start_station'], as_index=False)['Demand_for_the_period'].sum()
afternoon_outflow = afternoon_outflow.rename(columns={'Demand_for_the_period': 'outflow_demand'})

In [ ]:
morning_outflow.head()
afternoon_outflow.head()

In [ ]:
morning_inflow = morning_rush.groupby(['end_station'], as_index=False)['Demand_for_the_period'].sum()
morning_inflow = morning_inflow.rename(columns={'demand_rate': 'inflow_supply'})

afternoon_inflow = afternoon_rush.groupby(['end_station'], as_index=False)['Demand_for_the_period'].sum()
afternoon_inflow = afternoon_inflow.rename(columns={'demand_rate': 'inflow_supply'})

In [ ]:
morning_rush_merged = pd.merge(morning_outflow, morning_inflow, how="outer", left_on=['start_station'], right_on = ['end_station'])

afternoon_rush_merged = pd.merge(afternoon_outflow, afternoon_inflow, how="outer", left_on=['start_station'], right_on = ['end_station'])

In [ ]:
morning_rush_merged.head(10)
afternoon_rush_merged.head(10)

In [ ]:
#check to ensure the merge was done correctly
morning_rush_merged['start_station']==morning_rush_merged['end_station']

afternoon_rush_merged['start_station']==afternoon_rush_merged['end_station']

In [ ]:
morning_rush_merged = morning_rush_merged.rename(columns={'start_station': 'STATION'})
morning_rush_merged = morning_rush_merged.drop(columns=['end_station'])

afternoon_rush_merged = afternoon_rush_merged.rename(columns={'start_station': 'STATION'})
afternoon_rush_merged = afternoon_rush_merged.drop(columns=['end_station'])

In [ ]:
morning_rush_merged.info()

afternoon_rush_merged.info()

In [ ]:
morning_rush_merged["Imbalance_rate"] = abs(morning_rush_merged["outflow_demand"] - morning_rush_merged["Demand_for_the_period"])

afternoon_rush_merged["Imbalance_rate"] = abs(afternoon_rush_merged["outflow_demand"] - afternoon_rush_merged["Demand_for_the_period"])

In [ ]:
morning_rush_merged["Imbalance_rate_daily"] = abs(morning_rush_merged["outflow_demand"] - morning_rush_merged["Demand_for_the_period"])/15

afternoon_rush_merged["Imbalance_rate_daily"] = abs(afternoon_rush_merged["outflow_demand"] - afternoon_rush_merged["Demand_for_the_period"])/15

In [ ]:
morning_rush_merged.sort_values(by ='Imbalance_rate', ascending = False)


In [ ]:
afternoon_rush_merged.sort_values(by ='Imbalance_rate', ascending = False)

In [ ]:
morning = morning_rush_merged.drop([4])

In [ ]:
morning.sort_values(by ='Imbalance_rate', ascending = False)

Question (c)

In [ ]:
ride_data_group.head()

In [ ]:
time_spent_df = ride_data_group[['pair','Duration']]
time_spent_df.info()

In [ ]:
time_spent_mean = time_spent_df.groupby(['pair'], as_index=False)['Duration'].mean()
time_spent_mean 

In [ ]:
time_spent_sd = time_spent_df.groupby(['pair'], as_index=False)['Duration'].std()
time_spent_sd

In [ ]:
time_spend_final = pd.merge(time_spent_mean , time_spent_sd, how="outer", left_on=['pair'], right_on = ['pair'])

In [ ]:
time_spend_final.head()

In [ ]:
time_spend_final = time_spend_final.rename(columns={'Duration_x': 'Mean_(sec)', 'Duration_y': 'Standard_Deviation_(sec)'})

In [ ]:
time_spend_final

In [ ]:
time_spend_final[['start_station', 'end_station']] = time_spend_final['pair'].str.split('; ', 1, expand=True)

In [ ]:
time_spend_final = time_spend_final[["start_station", "end_station", "Mean_(sec)", "Standard_Deviation_(sec)", "pair"]]

In [ ]:
time_spend_final.head(10)

In [ ]:
cycling_times_from_HP_Corner = time_spend_final[time_spend_final['start_station'] == 'Hyde Park Corner, Hyde Park']
cycling_times_from_HP_Corner

Question (E)

In [ ]:
van_driving_mean= van_driving.drop(columns = ["drivingTime_std"])

In [ ]:
matrix = pd.pivot_table(van_driving_mean, values = 'drivingTime_mean', index=['StationA'], columns = 'StationB').fillna(0)
matrix

In [ ]:
van_driving.head(10)

In [ ]:
van_driving_mean= van_driving.drop(columns = ["drivingTime_std"])

In [ ]:
matrix = pd.pivot_table(van_driving_mean, values = 'drivingTime_mean', index=['StationA'], columns = 'StationB').fillna(0)
matrix

In [ ]:
values = matrix.values
values = values.tolist()
values

In [ ]:
!pip install --upgrade --user ortools

In [ ]:
!pip install --upgrade --user ortools

In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['duration_matrix'] = [
        [0,0,0,0,0,0,0,0,0],
        [0,0.0, 20.0, 7.0, 6.0, 14.0, 19.0, 22.0, 8.0],
        [0,20.0, 0.0, 17.0, 19.0, 10.0, 25.0, 23.0, 16.0],
        [0,7.0, 17.0, 0.0, 5.0, 12.0, 20.0, 22.0, 5.0],
        [0,6.0, 19.0, 5.0, 0.0, 13.0, 19.0, 20.0, 6.0],
        [0,14.0, 10.0, 12.0, 13.0, 0.0, 23.0, 24.0, 13.0],
        [0,19.0, 25.0, 20.0, 19.0, 23.0, 0.0, 4.0, 20.0],
        [0,22.0, 23.0, 22.0, 20.0, 24.0, 4.0, 0.0, 22.0],
        [0,8.0, 16.0, 5.0, 6.0, 13.0, 20.0, 22.0, 0.0],
    ] #manually adding the dummy values
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    max_route_duration = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_duration = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_duration += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Duration of the route: {}min\n'.format(route_duration)
        print(plan_output)
        max_route_duration = max(route_duration, max_route_duration)
    print('Maximum of the route durations: {}min'.format(max_route_duration))



def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['duration_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def duration_callback(from_index, to_index):
        """Returns the duration between the two nodes."""
        # Convert from routing variable Index to duration matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['duration_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(duration_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add duration constraint.
    dimension_name = 'duration'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        3000,  # vehicle maximum travel duration
        True,  # start cumul to zero
        dimension_name)
    duration_dimension = routing.GetDimensionOrDie(dimension_name)
    duration_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print('No solution found !')


if __name__ == '__main__':
    main()

Question (F) 

In [ ]:
class Bike_Station:
    '''Class to define the data structure of the bike station'''
    def __init__(self, env, num_dock, initial_bike, station_name):
        self.bike = simpy.Container(env, capacity = num_dock, init = initial_bike) # Num of bikes available initially
        self.user_dissatisfaction = 0 # User dissatisfaction incident count
        self.num_dock = num_dock # Num of docks for the station
        self.station_name = station_name # Name of the station
        self.user_start = 0 # Count of the user starts the journey at the station
        self.user_end = 0 # Count of the user ends the journey at the station

In [ ]:
def user(env, startStation_idx, endStation_idx):
    '''
    Function for simulating single user journey
    Arguments:
    startStation_idx: The index of the starting station in the station list, also corresponds to the index in the 
    riding time mean/std arrays
    startStation_idx: The index of the destination station in the station list, also corresponds to the index in 
    the riding time mean/std arrays'''
    start_station = stations[startStation_idx] # Get the bike station class from the station list
    end_station = stations[endStation_idx] # Get the bike station class from the station list
    start_station.user_start+=1 # Count of the user starts from the station +1
    if start_station.bike.level == 0: 
        # Check if there's no bike available at the station, add 1 to the count of dissatisfaction if that's the case
        print(f'Didn\'t get bike from {start_station.station_name} at {env.now:.1f}')
        start_station.user_dissatisfaction+=1
    else:
        print(f'Get bike from {start_station.station_name} at {env.now:.10f}')
        yield start_station.bike.get(1) # Get 1 bike from the station
        
        # Randomly generate the travel time based on a normal distribution
        travel_time = random.gauss(ridingTime_mean[startStation_idx][endStation_idx], 
                                   ridingTime_std[startStation_idx][endStation_idx])
        
        # The Randomly generated travel time should larger or equal to the minimum value in the historical data
        travel_time = max(travel_time, ridingTime_min[startStation_idx][endStation_idx])
        
        yield env.timeout(travel_time) # User rides the bike for X mins 
        print(f'User arrived at {end_station.station_name} at {env.now:.1f}')
        end_station.user_end += 1 # Count of the user arrives at the station +1
        
        start_wait_time = env.now # Record the time when the user arrives at the station
        park_bike = end_station.bike.put(1) # User tries to park the bike at the station
        yield park_bike # Yield the process as we wait for it to be finished
        if park_bike.triggered: # Check if the parking process is successfully triggered (finished)
            end_wait_time = env.now # Record the time when the user finished parking the bike
            if end_wait_time - start_wait_time > 0: 
                # Add 1 to the count of dissatisfaction if the user has waited for some time to park the bike
                end_station.user_dissatisfaction += 1
                print(f'Client dissatisfied, waiting time: {end_wait_time - start_wait_time:.1f}')
            #else:
                #print(f'Client satisfied, waiting time: {end_wait_time - start_wait_time:.1f}')

In [ ]:
def user_gen(env, startTime, endTime, arrivalRate, startStation_idx):
    '''
    Function for simulating user arrivals at a single station
    Arguments:
    startTime: Start time of the generation
    endTime: end time of the generation
    arrivalRate: the arrival rate of the user - used to generate inter-arrival time
    startStation_idx: the index of the start station in the station list'''
    while True: # Loop for waiting until the start of the process
        if env.now == startTime: # Start user arrival generation
            next_arrival = random.expovariate(arrivalRate) # Time interval 
            while env.now + next_arrival <= endTime: # The function would generate new arrival only before the end time
                yield env.timeout(next_arrival) # Time delay 
                
                # Randomly select a destination station based on the probability array
                endStation_idx = random.choices(range(len(stations)), weights=dest_prob[startStation_idx], k=1)[0]
                env.process(user(env, startStation_idx, endStation_idx)) # Start a single user journey simulation
                next_arrival = random.expovariate(arrivalRate)
            break # Prevent infinite loop
        yield env.timeout(1) # Time delay for 1 unit it reaches the start time

In [ ]:
class Rebalance_Van:
    '''Class to define the van for a full cycle of redistribution trip under a certain policy'''
    def __init__(self, env, capacity, initial_bike, startTime, route):
        self.capacity = capacity # Num of maximum bikes that the van can hold
        self.route = route # Route of the redistribution trip, a list of station indices indicating the sequence of station visits
        self.startTime = startTime # The time when the van starts the trip
        self.van = simpy.Container(env, capacity = capacity, init = initial_bike) # Container for simulating the storage on the van 
        self.van_routing = env.process(self.van_routing(env)) # Initialize the redistribution trip
        
        
    def van_routing(self, env):
        while True:
            yield env.timeout(1)
            if env.now == self.startTime:
                # We assume the van starts the trip at a bike station
                print(f'Van arrived at station {stations[self.route[0]].station_name} at {env.now:.1f}')
                
                # Rebalancing process according to the defined policy
                yield env.process(self.rebalance_process(env, stations[self.route[0]]))
                
                for idx in range(len(self.route)-1):
                    
                    # Generating travel time from the current station to the next one based on the mean and std,
                    # need to be at least 1 minute
                    yield env.timeout(max(random.gauss(vanRidingTime_mean[self.route[idx]][self.route[idx+1]], 
                                                   vanRidingTime_std[self.route[idx]][self.route[idx+1]]),1))
                    print(f'Van arrived at station {stations[self.route[idx+1]].station_name} at {env.now:.1f}')
                    yield env.process(self.rebalance_process(env, stations[self.route[idx+1]]))
                break
                
    def rebalance_process(self, env, station):
        # Rebalance policy - make the num of bikes to half of the capacity of the station
        # You can implement other rebalance policy by amending the code
        if station.bike.level > int(station.num_dock/2): # If there are more bikes than half of the capacity
            
            # Decide the num of bikes moving to the van from the station
            # Choose the smaller value between the number of bikes exceeding half of the capacity and available slots on the van
            num_bike_to_van = min(station.bike.level - int(station.num_dock/2), self.capacity - self.van.level)
            if num_bike_to_van>0: # Make sure the num get from or put to the container is larger than 0
                yield station.bike.get(num_bike_to_van) # Get bikes from the station
                yield env.timeout(num_bike_to_van*15/60) # Time spent on moving the bike, 15 seconds each (converted to minutes)
                print(f'Van get {num_bike_to_van} bikes from {station.station_name}')
        else:
            # Decide the num of bikes moving to the station from the van
            # Choose the smaller value between the number of bikes needed for filling half of the capacity and available bikes on the van
            num_bike_from_van = min(int(station.num_dock/2) - station.bike.level, self.van.level)
            if num_bike_from_van>0:
                yield station.bike.put(num_bike_from_van) # Put bikes to the station
                yield self.van.get(num_bike_from_van) # Get bikes from the van
                yield env.timeout(num_bike_from_van*15/60) # Time spent on moving the bikes
                print(f'Van put {num_bike_from_van} bikes to {station.station_name}')

In [ ]:
class dataGen:
    '''Class to record data during the simulation. The data can be used for visualization'''
    def __init__(self, env, startTime, endTime):
        self.startTime = startTime # Start time of recording data
        self.endTime = endTime # End time
        self.numBikes_list = [[] for i in range(len(stations))] # Initialize the list for recording the num of bikes at each station 
        self.dissatisfaction_list = [[] for i in range(len(stations))] # Initialize the list for recording the num of dissatisfaction incidents of each station 
        self.dataGen_process = env.process(self.dataGen(env)) # Start data collection process
        
    def dataGen(self, env):
        '''Function for data collection process'''
        while True: 
            # Loop until the start time
            if env.now == self.startTime:
                while True:
                    # Append the num of bikes at each station to the respective sublist
                    self.numBikes_list = [self.numBikes_list[idx]+[stations[idx].bike.level] 
                                          for idx in range(len(stations))]
                    
                    # Append the num of dissatisfaction incidents for each station to the respective sublist
                    self.dissatisfaction_list = [self.dissatisfaction_list[idx]+[
                        stations[idx].user_dissatisfaction] for idx in range(len(stations))]
                    yield env.timeout(1) # Record data every 1 unit time
                    if env.now == self.endTime: 
                        break # Terminate the collection process when reaches the end time
                break # Prevent infinite loop
            yield env.timeout(1)

In [ ]:
# Define the simulation environment
env = simpy.Environment()

In [ ]:
stations = [Bike_Station(env, num_dock = 34, initial_bike = 17, station_name = 'Albert Gate, Hyde Park'),
            Bike_Station(env, num_dock = 27, initial_bike = 14, station_name = 'Bayswater Road, Hyde Park'),
            Bike_Station(env, num_dock = 36, initial_bike = 18, station_name = 'Hyde Park Corner, Hyde Park'),
            Bike_Station(env, num_dock = 43, initial_bike = 21, station_name = 'Knightsbridge, Hyde Park'),
            Bike_Station(env, num_dock = 28, initial_bike = 14, station_name = 'Park Lane , Hyde Park'),
            Bike_Station(env, num_dock = 18, initial_bike = 9, station_name = 'Serpentine Car Park, Hyde Park'),
            Bike_Station(env, num_dock = 27, initial_bike = 13, station_name = 'Triangle Car Park, Hyde Park'),
            Bike_Station(env, num_dock = 36, initial_bike = 18, station_name = 'Wellington Arch, Hyde Park'),
            Bike_Station(env, num_dock = 1000, initial_bike = 500, station_name = 'Other')]
    

In [ ]:
#Taking Q(c) dataframe with Mean, Std, putting probability and Min into it to create time_spent_min~

In [ ]:
time_spent_min = time_spent_df.groupby(['pair'], as_index=False)['Duration'].min()
time_spent_min

In [ ]:
time_spend_final1 = pd.merge(time_spend_final, time_spent_min, how="outer", left_on=['pair'], right_on = ['pair'])

In [ ]:
time_spend_final1 = time_spend_final1.rename(columns={'Duration': 'Min_(sec)'})

In [ ]:
Dest_P = final_df[["pair","Probability"]]

In [ ]:
Dest_P.head(50)

In [ ]:
time_spend_final1 = pd.merge(time_spend_final1, Dest_P, how="outer", left_on=['pair'], right_on = ['pair'])

In [ ]:
time_spend_final1 = time_spend_final1.replace('Other','z_Other')


In [ ]:
time_spend_final1 = time_spend_final1.sort_values(by=["start_station"])

In [ ]:
time_spend_final1["start_station"].unique()

In [ ]:
time_spend_final1["Mean_min"] = time_spend_final1["Mean_(sec)"]/60
time_spend_final1["Standard_Deviation_min"] = time_spend_final1["Standard_Deviation_(sec)"]/60
time_spend_final1["Min_min"] = time_spend_final1["Min_(sec)"]/60

In [ ]:
time_spend_final1["start_station"].unique()

In [ ]:
time_spend_final1

In [ ]:
#Creating Matrix for all 4 values - mean, std, min, probability
#Creating Matrix for Van_driving_std, van_driving_mean

In [ ]:
ride_time_mean_matrix = pd.pivot_table(time_spend_final1, values = 'Mean_min', index=['start_station'], columns = 'end_station').fillna(0)
ride_time_std_matrix = pd.pivot_table(time_spend_final1, values = 'Standard_Deviation_min', index=['start_station'], columns = 'end_station').fillna(0)
ride_time_min_matrix = pd.pivot_table(time_spend_final1, values = 'Min_min', index=['start_station'], columns = 'end_station').fillna(0)
ride_time_P_matrix = pd.pivot_table(time_spend_final1, values = 'Probability', index=['start_station'], columns = 'end_station').fillna(0)

van_drivingTime_mean_Matrix = pd.pivot_table(van_driving, values = 'drivingTime_mean', index=['StationA'], columns = 'StationB').fillna(0)
van_drivingTime_std_Matrix = pd.pivot_table(van_driving, values = 'drivingTime_std', index=['StationA'], columns = 'StationB').fillna(0)

In [ ]:
ride_time_P_matrix 

In [ ]:
ride_time_mean_matrix_values = ride_time_mean_matrix.values
list_mean_ridetime = ride_time_mean_matrix_values.tolist()

ride_time_std_matrix_values = ride_time_std_matrix.values
list_std_ridetime = ride_time_std_matrix_values.tolist()

ride_time_min_matrix_values = ride_time_min_matrix.values
list_min_ridetime = ride_time_min_matrix_values.tolist()

ride_time_P_matrix_values = ride_time_P_matrix.values
list_P_ridetime = ride_time_P_matrix_values.tolist()

van_drivingTime_mean_Matrix_values = van_drivingTime_mean_Matrix.values
list_van_driving_mean = van_drivingTime_mean_Matrix_values.tolist()

van_drivingTime_std_Matrix_values = van_drivingTime_std_Matrix.values
list_van_driving_std = van_drivingTime_std_Matrix_values.tolist()

In [ ]:
list_mean_ridetime 

In [ ]:
stopper 

In [ ]:
# Hand defined synthetic data for the mean/std riding time, destination probabilities, minimum riding time, 
# mean/std van travelling time.
# Data defined in 2-D arrays.
# For example in the mean riding time array ridingTime_mean, ridingTime_mean[0][1] is the mean riding time from 
# station 0 (index in the list "stations") to station 1.
ridingTime_mean = np.array(list_mean_ridetime)
ridingTime_std = np.array(list_std_ridetime)
ridingTime_min = np.array(list_min_ridetime)
dest_prob = np.array(list_P_ridetime)

vanRidingTime_mean = np.array(list_van_driving_mean)
vanRidingTime_std = np.array(list_van_driving_std)

In [ ]:
ridingTime_min

In [ ]:
ridingTime_std

In [ ]:
arrival_df = final_df[["start_station", "start_hour", "Demand_per_startstation_at_certain_hr"]]

In [ ]:
arrival_df= arrival_df.drop_duplicates()

In [ ]:
arrival_df["Demand_per_day_per_mins"] = arrival_df["Demand_per_startstation_at_certain_hr"]/15/60

In [ ]:
arrival_df = arrival_df.replace('Other','z_Other')


In [ ]:
arrival_df = arrival_df.sort_values(by=["start_station"])

In [ ]:
#arrival_df= arrival_df.loc[arrival_df["start_station"] != "Other"]

In [ ]:
arrival_df["start_station"].unique()

In [ ]:
#arrival_df_matrix = pd.pivot_table(arrival_df, values = 'Demand_per_day_per_mins', index=['start_hour'], columns = 'start_station').fillna(0)

In [ ]:
arrival_df_matrix = pd.pivot_table(arrival_df, values = 'Demand_per_day_per_mins', index=['start_station'], columns ='start_hour').fillna(0)

In [ ]:
arrival_df_matrix

In [ ]:
arrival_df_matrix_values = arrival_df_matrix.values
list_arrival_df= arrival_df_matrix_values.tolist()

In [ ]:
arrival_rate = np.array(list_arrival_df)

In [ ]:
# Generate users
for station_idx in range(len(stations)):
    # Total period of 16 hours
    for period in range(16):
        env.process(user_gen(env, 
                             startTime = 60*period, 
                             endTime = 60*(period+1), 
                             arrivalRate = arrival_rate[station_idx][period], 
                             startStation_idx = station_idx))

In [ ]:
# Implement one rebalancing starts at 10 and the sequence of visiting is 1, 2, 3 (we don't put 0 as 
# that's the grouping of all the stations out of this scope).
rebalancing_1 = Rebalance_Van(env, capacity = 30, initial_bike = 20, startTime = 60, route = [1,4,7,2,3,0,5,6])


In [ ]:
# Record data for visulization (16*60)
recorded = dataGen(env, startTime = 0, endTime = 960)

# Run the simulation
env.run()

# Print the dissatisfaction of each station (you might need to run many simulations and calcluate the mean)
for station in stations:
    print(f'Num of dissatisfaction at station {station.station_name} is {station.user_dissatisfaction}')

In [ ]:
plt.figure(figsize=(10,5))
for i in range(0, 8):
    plt.plot(recorded.numBikes_list[i], label = stations[i].station_name)
plt.legend()
plt.title('Num of Bikes at each Station')
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
for i in range(0, 8):
    plt.plot(recorded.dissatisfaction_list[i], label = stations[i].station_name)
plt.title('Num of dissatisfaction records at each Station')
plt.legend()
plt.show()

Question G

Rebalance 1st

We changed the initial bike level for each station to the third of the available docks.

In [ ]:
# Define the simulation environment
env = simpy.Environment()

In [ ]:
stations = [Bike_Station(env, num_dock = 34, initial_bike = 22, station_name = 'Albert Gate, Hyde Park'),
            Bike_Station(env, num_dock = 27, initial_bike = 14, station_name = 'Bayswater Road, Hyde Park'),
            Bike_Station(env, num_dock = 36, initial_bike = 12, station_name = 'Hyde Park Corner, Hyde Park'),
            Bike_Station(env, num_dock = 43, initial_bike = 16, station_name = 'Knightsbridge, Hyde Park'),
            Bike_Station(env, num_dock = 28, initial_bike = 8, station_name = 'Park Lane , Hyde Park'),
            Bike_Station(env, num_dock = 18, initial_bike = 6, station_name = 'Serpentine Car Park, Hyde Park'),
            Bike_Station(env, num_dock = 27, initial_bike = 9, station_name = 'Triangle Car Park, Hyde Park'),
            Bike_Station(env, num_dock = 36, initial_bike = 12, station_name = 'Wellington Arch, Hyde Park'),
            Bike_Station(env, num_dock = 1000, initial_bike = 500, station_name = 'Other')]
    

In [ ]:
# Generate users
for station_idx in range(len(stations)):
    # Total period of 16 hours
    for period in range(16):
        env.process(user_gen(env, 
                             startTime = 60*period, 
                             endTime = 60*(period+1), 
                             arrivalRate = arrival_rate[station_idx][period], 
                             startStation_idx = station_idx))

In [ ]:
# Implement one rebalancing starts at 10 and the sequence of visiting is 1, 2, 3 (we don't put 0 as 
# that's the grouping of all the stations out of this scope).
rebalancing_2 = Rebalance_Van(env, capacity = 30, initial_bike = 20, startTime = 60, route = [1,4,7,2,3,0,5,6])


In [ ]:
# Record data for visulization (16*60)
recorded = dataGen(env, startTime = 0, endTime = 1020)

# Run the simulation
env.run()

# Print the dissatisfaction of each station (you might need to run many simulations and calcluate the mean)
for station in stations:
    print(f'Num of dissatisfaction at station {station.station_name} is {station.user_dissatisfaction}')

In [ ]:
plt.figure(figsize=(10,5))
for i in range(0, 8):
    plt.plot(recorded.numBikes_list[i], label = stations[i].station_name)
plt.legend()
plt.title('Num of Bikes at each Station')
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
for i in range(0, 8):
    plt.plot(recorded.dissatisfaction_list[i], label = stations[i].station_name)
plt.title('Num of dissatisfaction records at each Station')
plt.legend()
plt.show()

Simulation 2

From the previous simulation, Bayswater and Serpentine Car Park stations were very dissatisfied with fewer bikes so we increased the level of initial bikes at the station for them to a higher number.

In [ ]:
# Define the simulation environment
env = simpy.Environment()

In [ ]:
stations = [Bike_Station(env, num_dock = 34, initial_bike = 11, station_name = 'Albert Gate, Hyde Park'),
            Bike_Station(env, num_dock = 27, initial_bike = 18, station_name = 'Bayswater Road, Hyde Park'),
            Bike_Station(env, num_dock = 36, initial_bike = 12, station_name = 'Hyde Park Corner, Hyde Park'),
            Bike_Station(env, num_dock = 43, initial_bike = 16, station_name = 'Knightsbridge, Hyde Park'),
            Bike_Station(env, num_dock = 28, initial_bike = 8, station_name = 'Park Lane , Hyde Park'),
            Bike_Station(env, num_dock = 18, initial_bike = 6, station_name = 'Serpentine Car Park, Hyde Park'),
            Bike_Station(env, num_dock = 27, initial_bike = 18, station_name = 'Triangle Car Park, Hyde Park'),
            Bike_Station(env, num_dock = 36, initial_bike = 12, station_name = 'Wellington Arch, Hyde Park'),
            Bike_Station(env, num_dock = 1000, initial_bike = 500, station_name = 'Other')]
    

In [ ]:
# Generate users
for station_idx in range(len(stations)):
    # Total period of 16 hours
    for period in range(16):
        env.process(user_gen(env, 
                             startTime = 60*period, 
                             endTime = 60*(period+1), 
                             arrivalRate = arrival_rate[station_idx][period], 
                             startStation_idx = station_idx))

In [ ]:
# Implement one rebalancing starts at 10 and the sequence of visiting is 1, 2, 3 (we don't put 0 as 
# that's the grouping of all the stations out of this scope).
rebalancing_1 = Rebalance_Van(env, capacity = 30, initial_bike = 20, startTime = 60, route = [1,4,7,2,3,0,5,6])
rebalancing_2 = Rebalance_Van(env, capacity = 30, initial_bike = 20, startTime = 60, route = [1,4,7,2,3,0,5,6])

In [ ]:
# Record data for visulization (16*60)
recorded = dataGen(env, startTime = 0, endTime = 960)

# Run the simulation
env.run()

# Print the dissatisfaction of each station (you might need to run many simulations and calcluate the mean)
for station in stations:
    print(f'Num of dissatisfaction at station {station.station_name} is {station.user_dissatisfaction}')

In [ ]:
plt.figure(figsize=(10,5))
for i in range(0, 8):
    plt.plot(recorded.numBikes_list[i], label = stations[i].station_name)
plt.legend()
plt.title('Num of Bikes at each Station')
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
for i in range(0, 8):
    plt.plot(recorded.dissatisfaction_list[i], label = stations[i].station_name)
plt.title('Num of dissatisfaction records at each Station')
plt.legend()
plt.show()

Simulation 3

We changed the initial bike from 20 to 30 in the first rebalancing strategy. We kept the initial bike levels for each station from the previous simulation.

In [ ]:
# Define the simulation environment
env = simpy.Environment()

In [ ]:
stations = [Bike_Station(env, num_dock = 34, initial_bike = 11, station_name = 'Albert Gate, Hyde Park'),
            Bike_Station(env, num_dock = 27, initial_bike = 18, station_name = 'Bayswater Road, Hyde Park'),
            Bike_Station(env, num_dock = 36, initial_bike = 12, station_name = 'Hyde Park Corner, Hyde Park'),
            Bike_Station(env, num_dock = 43, initial_bike = 16, station_name = 'Knightsbridge, Hyde Park'),
            Bike_Station(env, num_dock = 28, initial_bike = 8, station_name = 'Park Lane , Hyde Park'),
            Bike_Station(env, num_dock = 18, initial_bike = 6, station_name = 'Serpentine Car Park, Hyde Park'),
            Bike_Station(env, num_dock = 27, initial_bike = 18, station_name = 'Triangle Car Park, Hyde Park'),
            Bike_Station(env, num_dock = 36, initial_bike = 12, station_name = 'Wellington Arch, Hyde Park'),
            Bike_Station(env, num_dock = 1000, initial_bike = 500, station_name = 'Other')]
    

In [ ]:
# Generate users
for station_idx in range(len(stations)):
    # Total period of 16 hours
    for period in range(16):
        env.process(user_gen(env, 
                             startTime = 60*period, 
                             endTime = 60*(period+1), 
                             arrivalRate = arrival_rate[station_idx][period], 
                             startStation_idx = station_idx))

In [ ]:
# Implement one rebalancing starts at 10 and the sequence of visiting is 1, 2, 3 (we don't put 0 as 
# that's the grouping of all the stations out of this scope).
rebalancing_1 = Rebalance_Van(env, capacity = 30, initial_bike = 30, startTime = 60, route = [1,4,7,2,3,0,5,6])
rebalancing_2 = Rebalance_Van(env, capacity = 30, initial_bike = 20, startTime = 180, route = [1,4,7,2,3,0,5,6])

In [ ]:
# Record data for visulization (16*60)
recorded = dataGen(env, startTime = 0, endTime = 1020)

# Run the simulation
env.run()

# Print the dissatisfaction of each station (you might need to run many simulations and calcluate the mean)
for station in stations:
    print(f'Num of dissatisfaction at station {station.station_name} is {station.user_dissatisfaction}')

In [ ]:
plt.figure(figsize=(10,5))
for i in range(0, 8):
    plt.plot(recorded.numBikes_list[i], label = stations[i].station_name)
plt.legend()
plt.title('Num of Bikes at each Station')
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
for i in range(0, 8):
    plt.plot(recorded.dissatisfaction_list[i], label = stations[i].station_name)
plt.title('Num of dissatisfaction records at each Station')
plt.legend()
plt.show()